In [1]:
import chardet
import pandas as pd
import numpy as np
from scipy.spatial import distance

In [2]:
def open_with_auto_sensing_encoding_types(file_path):
    with open(file_path, 'rb') as f:
        rawdata = f.read()
        result = chardet.detect(rawdata)
        encoding_type = result['encoding']

    print(f"파일의 인코딩: {encoding_type}")
    # 감지된 인코딩으로 파일 읽기
    df = pd.read_csv(file_path, encoding=encoding_type)
    print(len(df))
    return df

In [3]:
population_per_age_per_city = "/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/키즈카페입지분석2/data/광진구_인구수_및키즈카페정보/인구관련/서울특별시 광진구_연령별인구현황_20241028.csv"
population_per_city = "/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/키즈카페입지분석2/data/광진구_인구수_및키즈카페정보/인구관련/서울특별시_광진구_인구현황_20231231.csv"

population_per_age_per_city_df = open_with_auto_sensing_encoding_types(population_per_age_per_city)
population_per_city_df = open_with_auto_sensing_encoding_types(population_per_city)

파일의 인코딩: EUC-KR
21
파일의 인코딩: EUC-KR
16


In [4]:
print(population_per_city_df)
print("행정동 개수 : ", len(population_per_city_df))

     행정기관       계  인구수(남)  인구수(여)  지역(구성비)  구성비(남)  구성비(여)     성비     세대수  \
0     광진구  335554  161277  174277   100.00   48.06   51.94  92.54  170077   
1   중곡제1동   15239    7215    8024     4.54    2.15    2.39  89.92    8651   
2   중곡제2동   20521    9639   10882     6.12    2.87    3.24  88.58   10572   
3   중곡제3동   15727    7761    7966     4.69    2.31    2.37  97.43    8518   
4   중곡제4동   27594   13441   14153     8.22    4.01    4.22  94.97   13066   
5      능동   11097    5085    6012     3.31    1.52    1.79  84.58    6480   
6   구의제1동   23127   11060   12067     6.89    3.30    3.60  91.65   13100   
7   구의제2동   25610   12219   13391     7.63    3.64    3.99  91.25   11485   
8   구의제3동   27840   13148   14692     8.30    3.92    4.38  89.49   12051   
9     광장동   33971   16440   17531    10.12    4.90    5.22  93.78   11855   
10  자양제1동   21833   10576   11257     6.51    3.15    3.35  93.95   11985   
11  자양제2동   23628   11467   12161     7.04    3.42    3.62  94.29   10913   

In [5]:

for i in range(16):
    print(population_per_city_df['행정기관'][i], ":" ,population_per_city_df['행정기관'][i])

광진구 : 광진구
중곡제1동 : 중곡제1동
중곡제2동 : 중곡제2동
중곡제3동 : 중곡제3동
중곡제4동 : 중곡제4동
능동 : 능동
구의제1동 : 구의제1동
구의제2동 : 구의제2동
구의제3동 : 구의제3동
광장동 : 광장동
자양제1동 : 자양제1동
자양제2동 : 자양제2동
자양제3동 : 자양제3동
자양제4동 : 자양제4동
화양동 : 화양동
군자동 : 군자동


In [6]:
print(population_per_age_per_city_df.head(3))
print("행정동 개수 : ", len(population_per_age_per_city_df.columns)-1)

      광진구  화양동  군자동  중곡1동  중곡2동  중곡3동  중곡4동   능동  구의1동  구의2동  구의3동   광장동  \
0    0~4세  164  318   264   302   272   454  220   365   528   640   660   
1    5~9세  162  396   256   489   288   525  229   467   772   867  1716   
2  10~14세  201  462   327   619   355   731  257   542  1017  1063  2752   

   자양1동  자양2동  자양3동  자양4동     합계  
0   293   531   798   370   6179  
1   398   641  1053   425   8684  
2   613   801  1155   433  11328  
행정동 개수 :  16


In [7]:
# ---------------------
# 1. 행정기관 이름 정리 (제 제거)
population_per_city_df['행정기관'] = population_per_city_df['행정기관'].str.replace("제", "", regex=False)

# ---------------------
# 2. 연령별 인구 데이터 전치 및 정리
age_df = population_per_age_per_city_df.set_index(population_per_age_per_city_df.columns[0]).T
age_df.reset_index(inplace=True)
age_df = age_df.rename(columns={'index': '행정기관'})

# "광진구" 행 제거 (전체합이므로 병합 시 제외)
age_df = age_df[age_df['행정기관'] != '광진구']

# ---------------------
# 3. 병합
merged_df = pd.merge(population_per_city_df, age_df, on='행정기관', how='inner')

# 결과 확인
merged_df


,행정기관,계,인구수(남),인구수(여),지역(구성비),구성비(남),구성비(여),성비,세대수,세대당인구,...,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95~99세,100세 이상
0,중곡1동,15239,7215,8024,4.54,2.15,2.39,89.92,8651,1.76,...,1236,1180,1023,637,529,388,175,59,10,1
1,중곡2동,20521,9639,10882,6.12,2.87,3.24,88.58,10572,1.94,...,1696,1690,1391,944,718,469,201,53,16,0
2,중곡3동,15727,7761,7966,4.69,2.31,2.37,97.43,8518,1.85,...,1352,1427,1173,729,644,434,194,45,13,0
3,중곡4동,27594,13441,14153,8.22,4.01,4.22,94.97,13066,2.11,...,2685,2612,2205,1453,1082,741,331,103,24,6
4,능동,11097,5085,6012,3.31,1.52,1.79,84.58,6480,1.71,...,708,686,606,429,329,268,101,30,6,1
5,구의1동,23127,11060,12067,6.89,3.30,3.60,91.65,13100,1.77,...,1754,1612,1304,816,681,481,247,71,18,5
6,구의2동,25610,12219,13391,7.63,3.64,3.99,91.25,11485,2.23,...,2226,2073,1816,1134,949,615,324,95,14,7
7,구의3동,27840,13148,14692,8.30,3.92,4.38,89.49,12051,2.31,...,2210,2066,1767,1134,825,582,336,117,31,3
8,광장동,33971,16440,17531,10.12,4.90,5.22,93.78,11855,2.87,...,2496,1992,1616,999,639,553,364,139,44,5
9,자양1동,21833,10576,11257,6.51,3.15,3.35,93.95,11985,1.82,...,1813,1729,1464,906,717,579,245,74,22,1


In [8]:
print(merged_df.head(3))

   행정기관      계  인구수(남)  인구수(여)  지역(구성비)  구성비(남)  구성비(여)     성비    세대수  세대당인구  \
0  중곡1동  15239    7215    8024     4.54    2.15    2.39  89.92   8651   1.76   
1  중곡2동  20521    9639   10882     6.12    2.87    3.24  88.58  10572   1.94   
2  중곡3동  15727    7761    7966     4.69    2.31    2.37  97.43   8518   1.85   

   ...  55~59세  60~64세  65~69세  70~74세  75~79세  80~84세  85~89세  90~94세  \
0  ...    1236    1180    1023     637     529     388     175      59   
1  ...    1696    1690    1391     944     718     469     201      53   
2  ...    1352    1427    1173     729     644     434     194      45   

   95~99세  100세 이상  
0      10        1  
1      16        0  
2      13        0  

[3 rows x 31 columns]


In [9]:
merged_df['지역(구성비)']

0      4.54
1      6.12
2      4.69
3      8.22
4      3.31
5      6.89
6      7.63
7      8.30
8     10.12
9      6.51
10     7.04
11     8.11
12     5.93
13     6.91
14     5.68
Name: 지역(구성비), dtype: float64

새로운 csv 만들어야한다.
|행정기관|총인구수|0~4세|5~9세|0~9세|세대당인구|

In [10]:
# 필요한 열만 추출
summary_df = merged_df[['행정기관', '계', '0~4세', '5~9세', '세대당인구']].copy()

# 열 이름 변경
summary_df = summary_df.rename(columns={
    '계': '총인구수',
})

# 0~9세 계산
summary_df['0~9세'] = summary_df['0~4세'] + summary_df['5~9세']

# 열 순서 정리
summary_df = summary_df[['행정기관', '총인구수', '0~4세', '5~9세', '0~9세', '세대당인구']]

# 결과 확인

summary_df
# # CSV 저장



,행정기관,총인구수,0~4세,5~9세,0~9세,세대당인구
0,중곡1동,15239,264,256,520,1.76
1,중곡2동,20521,302,489,791,1.94
2,중곡3동,15727,272,288,560,1.85
3,중곡4동,27594,454,525,979,2.11
4,능동,11097,220,229,449,1.71
5,구의1동,23127,365,467,832,1.77
6,구의2동,25610,528,772,1300,2.23
7,구의3동,27840,640,867,1507,2.31
8,광장동,33971,660,1716,2376,2.87
9,자양1동,21833,293,398,691,1.82


In [11]:
summary_df.to_csv("광진구_연령별_요약.csv", index=False, encoding='utf-8-sig')

In [12]:
print(summary_df.head(17))

    행정기관   총인구수  0~4세  5~9세  0~9세  세대당인구
0   중곡1동  15239   264   256   520   1.76
1   중곡2동  20521   302   489   791   1.94
2   중곡3동  15727   272   288   560   1.85
3   중곡4동  27594   454   525   979   2.11
4     능동  11097   220   229   449   1.71
5   구의1동  23127   365   467   832   1.77
6   구의2동  25610   528   772  1300   2.23
7   구의3동  27840   640   867  1507   2.31
8    광장동  33971   660  1716  2376   2.87
9   자양1동  21833   293   398   691   1.82
10  자양2동  23628   531   641  1172   2.17
11  자양3동  27221   798  1053  1851   2.48
12  자양4동  19883   370   425   795   1.81
13   화양동  23193   164   162   326   1.28
14   군자동  19070   318   396   714   1.69


# 1. 유치원_아동센터_어린이집 개수 동별 개수 추출

In [13]:
def open_with_auto_sensing_encoding_types(file_path):
    import chardet

    with open(file_path, 'rb') as f:
        rawdata = f.read()
        result = chardet.detect(rawdata)
        encoding_type = result['encoding']

    print(f"파일의 인코딩: {encoding_type}")
    
    try:
        # 에러 행 무시하고 불러오기
        df = pd.read_csv(file_path, encoding=encoding_type, error_bad_lines=False, engine='python')
    except Exception as e:
        print(f"CSV 파싱 중 오류 발생: {e}")
        return None

    print(f"{file_path} → 행 개수: {len(df)}, 열 개수: {len(df.columns)}")
    return df


In [14]:
daycare = "/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/키즈카페입지분석2/data/광진구_인구수_및키즈카페정보/유치원_아동센터_어린이집/서울시광진구어린이집정보표준데이터.csv"
kindergarten = "/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/키즈카페입지분석2/data/광진구_인구수_및키즈카페정보/유치원_아동센터_어린이집/서울시광진구유치원일반현황.csv"
children_center = "/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/키즈카페입지분석2/data/광진구_인구수_및키즈카페정보/유치원_아동센터_어린이집/서울특별시 광진구_지역아동센터_20250110.csv"

daycare_df = open_with_auto_sensing_encoding_types(daycare)
kindergarten_df = open_with_auto_sensing_encoding_types(kindergarten)
children_center_df = open_with_auto_sensing_encoding_types(children_center)

파일의 인코딩: EUC-KR
/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/키즈카페입지분석2/data/광진구_인구수_및키즈카페정보/유치원_아동센터_어린이집/서울시광진구어린이집정보표준데이터.csv → 행 개수: 205, 열 개수: 62
파일의 인코딩: EUC-KR
/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/키즈카페입지분석2/data/광진구_인구수_및키즈카페정보/유치원_아동센터_어린이집/서울시광진구유치원일반현황.csv → 행 개수: 38, 열 개수: 24
파일의 인코딩: EUC-KR
/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/키즈카페입지분석2/data/광진구_인구수_및키즈카페정보/유치원_아동센터_어린이집/서울특별시 광진구_지역아동센터_20250110.csv → 행 개수: 13, 열 개수: 14


/var/folders/6q/q927ypwn4ll9d4rvjq9gl1qw0000gn/T/ipykernel_66224/2831215575.py:13: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_path, encoding=encoding_type, error_bad_lines=False, engine='python')
Skipping line 8: Expected 62 fields in line 8, saw 63
Skipping line 15: Expected 62 fields in line 15, saw 63
Skipping line 17: Expected 62 fields in line 17, saw 63
Skipping line 21: Expected 62 fields in line 21, saw 63
Skipping line 22: Expected 62 fields in line 22, saw 63
Skipping line 25: Expected 62 fields in line 25, saw 63
Skipping line 28: Expected 62 fields in line 28, saw 63
Skipping line 30: Expected 62 fields in line 30, saw 63
Skipping line 31: Expected 62 fields in line 31, saw 63
Skipping line 33: Expected 62 fields in line 33, saw 63
Skipping line 35: Expected 62 fields in line 35, saw 63
Skipping line 36: Expected 62 fields in line 36, saw 63
Skipping line 

In [15]:
daycare_df.head(2)

,시군구명,어린이집코드,어린이집명,어린이집유형,운영현황,우편번호,상세주소,전화번호,팩스번호,홈페이지주소,...,교직원현황-보육교사,교직원현황-특수교사,교직원현황-치료교사,교직원현황-영양사,교직원현황-간호사,교직원현황-간호조무사,교직원현황-조리원,교직원현황-사무직원,교직원현황-총계,데이터수집일
0,광진구,11215000369,구립 자양이스트폴어린이집,국공립,정상,5050,서울특별시 광진구 뚝섬로 717 구립 자양이스트폴어린이집,02-450-7547,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2025-03-25 22:14:38.0
1,광진구,11215000368,구립 해달별어린이집,국공립,정상,5048,서울특별시 광진구 구의강변로3가길 39 701동 102호,02-455-7516,NaN,NaN,...,5.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,2025-03-25 22:14:38.0


In [16]:
print(kindergarten_df)
kindergarten_df.head(2)

        교육청명     교육지원청명                                 유치원코드           유치원명  \
0   서울특별시교육청  성동광진교육지원청  1ecec08c-eee1-b044-e053-0a32095ab044          선경유치원   
1   서울특별시교육청  성동광진교육지원청  1ecec08c-eee2-b044-e053-0a32095ab044          성산유치원   
2   서울특별시교육청  성동광진교육지원청  1ecec08c-eee3-b044-e053-0a32095ab044          자양유치원   
3   서울특별시교육청  성동광진교육지원청  1ecec08c-eee4-b044-e053-0a32095ab044         프라임유치원   
4   서울특별시교육청  성동광진교육지원청  1ecec08c-f0da-b044-e053-0a32095ab044  서울자양초등학교병설유치원   
5   서울특별시교육청  성동광진교육지원청  1ecec08c-f1c6-b044-e053-0a32095ab044          광남유치원   
6   서울특별시교육청  성동광진교육지원청  1ecec08c-f331-b044-e053-0a32095ab044       어린이회관유치원   
7   서울특별시교육청  성동광진교육지원청  1ecec08c-f332-b044-e053-0a32095ab044          영화유치원   
8   서울특별시교육청  성동광진교육지원청  1ecec08c-f504-b044-e053-0a32095ab044          동인유치원   
9   서울특별시교육청  성동광진교육지원청  1ecec08c-f562-b044-e053-0a32095ab044          예일유치원   
10  서울특별시교육청  성동광진교육지원청  1ecec08c-f7d7-b044-e053-0a32095ab044          슬기유치원   
11  서울특별시교육청  성동광진교육지원청  1ecec08c-f7d8-b

,교육청명,교육지원청명,유치원코드,유치원명,설립유형,대표자명,원장명,설립일,개원일,주소,...,만4세학급수,만5세학급수,혼합학급수,특수학급수,만3세유아수,만4세유아수,만5세유아수,혼합유아수,특수유아수,공시차수
0,서울특별시교육청,성동광진교육지원청,1ecec08c-eee1-b044-e053-0a32095ab044,선경유치원,사립(법인),성현정,성현정,19680308,19680308,서울특별시 광진구 뚝섬로66길 14,...,3.0,2.0,0,NaN,20.0,57.0,44.0,0,NaN,20242
1,서울특별시교육청,성동광진교육지원청,1ecec08c-eee2-b044-e053-0a32095ab044,성산유치원,사립(법인),대한예수교장로회고신총회유지재단,조은영,19840310,19840310,서울특별시 광진구 뚝섬로59길 18,...,1.0,1.0,0,NaN,11.0,14.0,17.0,0,NaN,20242


In [17]:
kindergarten_df.head(2)

,교육청명,교육지원청명,유치원코드,유치원명,설립유형,대표자명,원장명,설립일,개원일,주소,...,만4세학급수,만5세학급수,혼합학급수,특수학급수,만3세유아수,만4세유아수,만5세유아수,혼합유아수,특수유아수,공시차수
0,서울특별시교육청,성동광진교육지원청,1ecec08c-eee1-b044-e053-0a32095ab044,선경유치원,사립(법인),성현정,성현정,19680308,19680308,서울특별시 광진구 뚝섬로66길 14,...,3.0,2.0,0,NaN,20.0,57.0,44.0,0,NaN,20242
1,서울특별시교육청,성동광진교육지원청,1ecec08c-eee2-b044-e053-0a32095ab044,성산유치원,사립(법인),대한예수교장로회고신총회유지재단,조은영,19840310,19840310,서울특별시 광진구 뚝섬로59길 18,...,1.0,1.0,0,NaN,11.0,14.0,17.0,0,NaN,20242


# 2. 키즈카페 및 내부놀이터 개수 동별 개수 추출

In [18]:
indoor_playgrounds = "/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/키즈카페입지분석2/data/광진구_인구수_및키즈카페정보/광진구_키즈카페_목록.csv"
indoor_playgrounds_df = open_with_auto_sensing_encoding_types(indoor_playgrounds)
indoor_playgrounds_df.head(20)

파일의 인코딩: UTF-8-SIG
/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/키즈카페입지분석2/data/광진구_인구수_및키즈카페정보/광진구_키즈카페_목록.csv → 행 개수: 18, 열 개수: 3


/var/folders/6q/q927ypwn4ll9d4rvjq9gl1qw0000gn/T/ipykernel_66224/2831215575.py:13: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_path, encoding=encoding_type, error_bad_lines=False, engine='python')


,시설이름,시설구분,동이름
0,캘리클럽 강변점,"키즈카페,실내놀이터",서울 광진구 구의동
1,서울형키즈카페 시립뚝섬자벌레점,"키즈카페,실내놀이터",서울 광진구 자양동
2,펀노리터 서울광진점,"키즈카페,실내놀이터",서울 광진구 중곡동
3,데굴데굴키즈파티룸 구의점,"키즈카페,실내놀이터",서울 광진구 구의동
4,퐁퐁플라워 건대센터,"키즈카페,실내놀이터",서울 광진구 자양동
5,점핑몬스터 구의점,"키즈카페,실내놀이터",서울 광진구 구의동
6,더키즈프리미엄키즈카페,"키즈카페,실내놀이터",서울 광진구 중곡동
7,클래비키즈파크,"키즈카페,실내놀이터",서울 광진구 능동
8,프로맘킨더 롯데백화점 건대스타시티점,"키즈카페,실내놀이터",서울 광진구 자양동
9,서울형 키즈카페 광진구 중곡 3동점,"키즈카페,실내놀이터",서울 광진구 중곡동


In [19]:
# 1. "서울 광진구" 제거 → 동이름만 추출
indoor_playgrounds_df['동이름'] = indoor_playgrounds_df['동이름'].str.replace("서울 광진구 ", "", regex=False)



In [20]:
indoor_playgrounds_df['동이름']

0     구의동
1     자양동
2     중곡동
3     구의동
4     자양동
5     구의동
6     중곡동
7      능동
8     자양동
9     중곡동
10    자양동
11    군자동
12    광장동
13    자양점
14    구의동
15    중곡동
16    중곡동
17    자양동
Name: 동이름, dtype: object

In [21]:
# 2. 동별 시설 개수 집계
facility_counts = indoor_playgrounds_df['동이름'].value_counts().to_dict()
print(facility_counts)  # 예: {'자양동': 3, '구의동': 2, ...}

{'자양동': 5, '중곡동': 5, '구의동': 4, '능동': 1, '군자동': 1, '광장동': 1, '자양점': 1}


In [22]:
# 3. 법정동 → 행정동 매핑 딕셔너리
법정동_to_행정동 = {
    '중곡동': ['중곡1동', '중곡2동', '중곡3동', '중곡4동'],
    '구의동': ['구의1동', '구의2동', '구의3동'],
    '자양동': ['자양1동', '자양2동', '자양3동', '자양4동'],
    '화양동': ['화양동'],
    '군자동': ['군자동'],
    '능동': ['능동'],
    '광장동': ['광장동'],
}

# 4. summary_df에 "실내놀이터" 열 추가 (0으로 초기화)
summary_df['실내놀이터'] = 0

# 5. 매핑을 통해 시설 개수를 행정동에 더함
for 법정동, 행정동_리스트 in 법정동_to_행정동.items():
    count = facility_counts.get(법정동, 0)
    for 행정동 in 행정동_리스트:
        summary_df.loc[summary_df['행정기관'] == 행정동, '실내놀이터'] += count

# 결과 확인
print(summary_df[['행정기관', '실내놀이터']])


    행정기관  실내놀이터
0   중곡1동      5
1   중곡2동      5
2   중곡3동      5
3   중곡4동      5
4     능동      1
5   구의1동      4
6   구의2동      4
7   구의3동      4
8    광장동      1
9   자양1동      5
10  자양2동      5
11  자양3동      5
12  자양4동      5
13   화양동      0
14   군자동      1


In [23]:
print(summary_df.head(15))

    행정기관   총인구수  0~4세  5~9세  0~9세  세대당인구  실내놀이터
0   중곡1동  15239   264   256   520   1.76      5
1   중곡2동  20521   302   489   791   1.94      5
2   중곡3동  15727   272   288   560   1.85      5
3   중곡4동  27594   454   525   979   2.11      5
4     능동  11097   220   229   449   1.71      1
5   구의1동  23127   365   467   832   1.77      4
6   구의2동  25610   528   772  1300   2.23      4
7   구의3동  27840   640   867  1507   2.31      4
8    광장동  33971   660  1716  2376   2.87      1
9   자양1동  21833   293   398   691   1.82      5
10  자양2동  23628   531   641  1172   2.17      5
11  자양3동  27221   798  1053  1851   2.48      5
12  자양4동  19883   370   425   795   1.81      5
13   화양동  23193   164   162   326   1.28      0
14   군자동  19070   318   396   714   1.69      1


In [24]:
행정동_to_법정동 = {
    '중곡1동': '중곡동',
    '중곡2동': '중곡동',
    '중곡3동': '중곡동',
    '중곡4동': '중곡동',
    '구의1동': '구의동',
    '구의2동': '구의동',
    '구의3동': '구의동',
    '자양1동': '자양동',
    '자양2동': '자양동',
    '자양3동': '자양동',
    '자양4동': '자양동',
    '화양동': '화양동',
    '군자동': '군자동',
    '능동': '능동',
    '광장동': '광장동',
}


In [25]:
# 행정동 → 법정동 컬럼 생성
summary_df['법정동'] = summary_df['행정기관'].map(행정동_to_법정동)

# 평균이 의미 있는 항목과 합계가 필요한 항목 구분
합계_항목 = ['총인구수', '0~4세', '5~9세', '0~9세']
유지_항목 = ['실내놀이터']
평균_항목 = ['세대당인구']

# 그룹화하여 집계
법정동_df = summary_df.groupby('법정동').agg(
    {**{col: 'sum' for col in 합계_항목},
     **{col: 'first' for col in 유지_항목},
     **{col: 'mean' for col in 평균_항목}}
).reset_index()

# 결과 확인
print(법정동_df)


   법정동   총인구수  0~4세  5~9세  0~9세  실내놀이터     세대당인구
0  광장동  33971   660  1716  2376      1  2.870000
1  구의동  76577  1533  2106  3639      4  2.103333
2  군자동  19070   318   396   714      1  1.690000
3   능동  11097   220   229   449      1  1.710000
4  자양동  92565  1992  2517  4509      5  2.070000
5  중곡동  79081  1292  1558  2850      5  1.915000
6  화양동  23193   164   162   326      0  1.280000


In [26]:
법정동_df["0~4세비율"] = 법정동_df["0~4세"] /법정동_df["총인구수"]
법정동_df["5~9세비율"] = 법정동_df["5~9세"] /법정동_df["총인구수"]
법정동_df["0~9세비율"] = 법정동_df["0~9세"] /법정동_df["총인구수"]
법정동_df["실내놀이터당아동"] = 법정동_df["0~9세"] /법정동_df["실내놀이터"]
# inf → NaN 변환
법정동_df.replace([np.inf, -np.inf], 0, inplace=True)

# NaN이 포함된 행 제거
법정동_df.dropna(subset=['실내놀이터당아동'], inplace=True)

In [27]:
법정동_df

,법정동,총인구수,0~4세,5~9세,0~9세,실내놀이터,세대당인구,0~4세비율,5~9세비율,0~9세비율,실내놀이터당아동
0,광장동,33971,660,1716,2376,1,2.870000,0.019428,0.050514,0.069942,2376.00
1,구의동,76577,1533,2106,3639,4,2.103333,0.020019,0.027502,0.047521,909.75
2,군자동,19070,318,396,714,1,1.690000,0.016675,0.020766,0.037441,714.00
3,능동,11097,220,229,449,1,1.710000,0.019825,0.020636,0.040461,449.00
4,자양동,92565,1992,2517,4509,5,2.070000,0.021520,0.027192,0.048712,901.80
5,중곡동,79081,1292,1558,2850,5,1.915000,0.016338,0.019701,0.036039,570.00
6,화양동,23193,164,162,326,0,1.280000,0.007071,0.006985,0.014056,0.00


In [28]:
법정동_df.to_csv("법정동단위통합파일.csv")